# Preprocess somatic mutations file

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import re
import numpy as np

Set the path to save output files (PATH) and folder with data files (DATAPATH)

In [2]:
PATH = "D:/Projects/cTaG2.0"
DATAPATH = "D:/Projects/data/cTaG2.0"

### Uploading the maf file to generate annovar input file
Upload the maf file

In [35]:
os.chdir(DATAPATH + "/GDC_TCGA/SNV/995c0111-d90b-4140-bee7-3845436c3b42")
fname = "TCGA.BRCA.mutect.995c0111-d90b-4140-bee7-3845436c3b42.DR-10.0.somatic.maf"
data_snp=pd.read_csv(fname, sep="\t", comment="#", low_memory=False)

In [36]:
data_snp.shape

(120988, 120)

In [37]:
print("Total number of samples = {}".format(len(data_snp.Tumor_Sample_Barcode.unique())))

Total number of samples = 986


In [38]:
data_snp.head()

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,...,FILTER,CONTEXT,src_vcf_id,tumor_bam_uuid,normal_bam_uuid,case_id,GDC_FILTER,COSMIC,MC3_Overlap,GDC_Validation_Status
0,USP24,23358,WUGSC,GRCh38,chr1,55159655,55159655,+,Missense_Mutation,SNP,...,panel_of_normals,CTGGATTGTAG,d083d669-6646-463b-853e-c58da8d06439,4374e19d-c5e7-49cf-8707-05ae5aeb7369,aadee87c-6a68-4580-bd10-64ac273b1e3d,0130d616-885e-4a6c-9d03-2f17dd692a05,common_in_exac;gdc_pon,NaN,True,Unknown
1,ERICH3,127254,WUGSC,GRCh38,chr1,74571494,74571494,+,Missense_Mutation,SNP,...,PASS,TTCCTCTACCA,d083d669-6646-463b-853e-c58da8d06439,4374e19d-c5e7-49cf-8707-05ae5aeb7369,aadee87c-6a68-4580-bd10-64ac273b1e3d,0130d616-885e-4a6c-9d03-2f17dd692a05,NaN,COSM1474194,True,Unknown
2,KIF26B,55083,WUGSC,GRCh38,chr1,245419680,245419680,+,Silent,SNP,...,PASS,GCCTCGCAGGG,d083d669-6646-463b-853e-c58da8d06439,4374e19d-c5e7-49cf-8707-05ae5aeb7369,aadee87c-6a68-4580-bd10-64ac273b1e3d,0130d616-885e-4a6c-9d03-2f17dd692a05,NaN,COSM1473725;COSM1473726,True,Unknown
3,USP34,9736,WUGSC,GRCh38,chr2,61189055,61189055,+,Silent,SNP,...,PASS,AAAGCGAGTGC,d083d669-6646-463b-853e-c58da8d06439,4374e19d-c5e7-49cf-8707-05ae5aeb7369,aadee87c-6a68-4580-bd10-64ac273b1e3d,0130d616-885e-4a6c-9d03-2f17dd692a05,NaN,COSM1483177,True,Unknown
4,ANTXR1,84168,WUGSC,GRCh38,chr2,69245305,69245305,+,Silent,SNP,...,PASS,TCCTCGCCGCC,d083d669-6646-463b-853e-c58da8d06439,4374e19d-c5e7-49cf-8707-05ae5aeb7369,aadee87c-6a68-4580-bd10-64ac273b1e3d,0130d616-885e-4a6c-9d03-2f17dd692a05,NaN,COSM1409122,True,Unknown


In [39]:
for col in data_snp.columns:
    print(col, data_snp.loc[0,col])

Hugo_Symbol USP24
Entrez_Gene_Id 23358
Center WUGSC
NCBI_Build GRCh38
Chromosome chr1
Start_Position 55159655
End_Position 55159655
Strand +
Variant_Classification Missense_Mutation
Variant_Type SNP
Reference_Allele T
Tumor_Seq_Allele1 T
Tumor_Seq_Allele2 C
dbSNP_RS rs150880897
dbSNP_Val_Status by1000G;byCluster;byFrequency
Tumor_Sample_Barcode TCGA-D8-A1XY-01A-11D-A14K-09
Matched_Norm_Sample_Barcode TCGA-D8-A1XY-10A-01D-A14K-09
Match_Norm_Seq_Allele1 nan
Match_Norm_Seq_Allele2 nan
Tumor_Validation_Allele1 nan
Tumor_Validation_Allele2 nan
Match_Norm_Validation_Allele1 nan
Match_Norm_Validation_Allele2 nan
Verification_Status nan
Validation_Status nan
Mutation_Status Somatic
Sequencing_Phase nan
Sequence_Source nan
Validation_Method nan
Score nan
BAM_File nan
Sequencer Illumina HiSeq 2000
Tumor_Sample_UUID edb6d161-8f50-4c11-8246-487c4ea9a55d
Matched_Norm_Sample_UUID 8dea96d9-5017-4872-a84e-33bfd2f37b7a
HGVSc c.1024A>G
HGVSp p.Ile342Val
HGVSp_Short p.I342V
Transcript_ID ENST00000294383


Generate input file for ANNOVAR

In [40]:
data_annovar = data_snp.loc[:,["Chromosome", "Start_Position", "End_Position", "Reference_Allele", "Tumor_Seq_Allele2"]]
data_annovar.Chromosome = [x[3:] for x in data_annovar.Chromosome]
data_annovar.head()

,Chromosome,Start_Position,End_Position,Reference_Allele,Tumor_Seq_Allele2
0,1,55159655,55159655,T,C
1,1,74571494,74571494,C,T
2,1,245419680,245419680,G,T
3,2,61189055,61189055,G,C
4,2,69245305,69245305,G,A


In [188]:
# check end positions
temp=[]
for s,e,r,a in zip(data_annovar.Start_Position, data_annovar.End_Position, data_annovar.Reference_Allele, data_annovar.Tumor_Seq_Allele2):
    if s == e and len(r) == 1:
        temp.append(True)
    elif e-s == len(r) - 1:
        temp.append(True)
    else:
        temp.append(False)
temp.count(False)
        


3739

In [189]:
len(data_annovar[data_annovar.Tumor_Seq_Allele2 == "-"])

5209

In [190]:
os.chdir(DATAPATH + "/GDC_TCGA/SNV")
fname="brca_snp.avinput"
data_annovar.to_csv(fname, sep="\t", header=False, index=False)

## Run ANNOVAR
run shell command to run annovar using the generated file as input

Go to folder containing annovar:
cd /data/Priyanka/softwares/annovar/

command:
table_annovar.pl /data/malvika/temp/brca_snp.avinput /data/malvika/humandb/ -buildver hg38 -out brca_anno.hg38_multianno.csv -remove -protocol refGene,cytoBand,exac03,avsnp147,dbnsfp30a -operation gx,r,f,f,f -nastring . -csvout -polish -xref example/gene_xref.txt

Copy the resultant file to relevant folder



## Concatenate the output file from ANNOVAR and maf file
Load ANNOVAR output file

In [41]:
os.chdir(DATAPATH + "/GDC_TCGA/SNV")
fname = "brca_anno.hg38_multiannov.hg38_multianno.csv"
data_anno=pd.read_csv(fname, sep=",", comment="#").drop(0).replace(".", np.nan).reset_index()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [42]:
data_all = pd.concat([data_snp, data_anno], axis=1)

### Discard hypermutated samples

In [43]:
max_mut = 2000
mut_count = data_all.groupby(["Tumor_Sample_Barcode"]).Tumor_Sample_Barcode.count()
data_all = data_all[~data_all.Tumor_Sample_Barcode.isin(mut_count[mut_count>2000].index)]

In [44]:
print("Total number of samples (after filtering) = {}".format(len(data_all.Tumor_Sample_Barcode.unique())))

Total number of samples (after filtering) = 984


## Covert the old gene symbols into relevant genes
Load hugo to ensemble id file and check for gene symbols not recognised.
Take unique list of genes and save to file.

In [45]:
# Load hugo to ensemble id mapping data
os.chdir("D:/Projects/data/cTaG2.0")
fname = "hugo2ensmbl.txt"
data_map = pd.read_csv(fname, sep="\t", header=0, index_col=1)


In [46]:
os.chdir(DATAPATH + "/GDC_TCGA/SNV")
fname = "convertgenes.txt"
pd.Series(list(set([x for x in data_all.Hugo_Symbol if x not in data_map.index]))).to_csv(fname, index=False, header=False)

Use HGNC converter
https://www.genenames.org/tools/multi-symbol-checker/
If the converter throws error or takes entire gene list as one entry, delete and re-enter first new line in the uploaded genelist and try again.

Download the output  as csv and open with notepad and delete first line from output csv file (sep=,).
The output file is now ready for use.

In [47]:
os.chdir(DATAPATH + "/GDC_TCGA/SNV")
fname = "hgnc-symbol-check.csv"
gene_map = pd.read_csv(fname, index_col=0, header=0)
gene_map = gene_map[gene_map["Match type"] != "Unmatched"]

In [48]:
gene_map.head()

,Match type,Approved symbol,Approved name,HGNC ID,Location
Input,,,,,
ERBB2IP,Previous symbol,ERBIN,erbb2 interacting protein,HGNC:15842,5q12.3
C5orf42,Previous symbol,CPLANE1,ciliogenesis and planar polarity effector 1,HGNC:25801,5p13.2
C7orf66,Previous symbol,LINC02903,long intergenic non-protein coding RNA 2903,HGNC:33712,7q31.1
NGFRAP1,Previous symbol,BEX3,brain expressed X-linked 3,HGNC:13388,Xq22.2
WAPAL,Previous symbol,WAPL,WAPL cohesin release factor,HGNC:23293,10q23.2


In [49]:
# Update the gene symbols in dataframe
for idx, gene_sym, gene_ensg in zip(data_all.index, data_all["Hugo_Symbol"], data_all["Gene"]):
    if gene_sym in gene_map.index:
        if type(gene_map.loc[gene_sym, "Approved symbol"]) is not str:
            data_all.loc[idx, "Hugo_Symbol"] = (data_map[data_map["Ensembl gene ID"] == gene_ensg].index[0])
        else:
            data_all.loc[idx, "Hugo_Symbol"] = gene_map.loc[gene_sym, "Approved symbol"]
        

In [50]:
len(set([x for x in data_all.Hugo_Symbol if x not in data_map.index]))

375

### Keep relevant columns

In [51]:
cols = ['Hugo_Symbol', "Gene", "Tumor_Sample_Barcode", "Chr", "Start", "End", "Ref",
        "Alt",'Variant_Classification', 'DOMAINS',
        'SIFT_score', 'SIFT_pred', 'Polyphen2_HDIV_score',
        'Polyphen2_HDIV_pred', 'Polyphen2_HVAR_score', 'Polyphen2_HVAR_pred',
        'LRT_score', 'LRT_pred', 'MutationTaster_score', 'MutationTaster_pred',
        'MutationAssessor_score', 'MutationAssessor_pred', 'FATHMM_score',
        'FATHMM_pred', 'PROVEAN_score', 'PROVEAN_pred', 'VEST3_score',
        'CADD_raw', 'CADD_phred', 'DANN_score', 'fathmm-MKL_coding_score',
        'fathmm-MKL_coding_pred', 'MetaSVM_score', 'MetaSVM_pred',
        'MetaLR_score', 'MetaLR_pred', 'integrated_fitCons_score',
        'integrated_confidence_value', 'GERP++_RS',
        'phyloP7way_vertebrate', 'phyloP20way_mammalian',
        'phastCons7way_vertebrate', 'phastCons20way_mammalian',
        'SiPhy_29way_logOdds']
data = data_all[cols]

Save file

In [166]:
os.chdir(DATAPATH + "/GDC_TCGA/SNV")
fname="brca_snv.tsv"
data.to_csv(fname, sep="\t", header=True, index=False)

In [167]:
data.head()

,Hugo_Symbol,Gene,Tumor_Sample_Barcode,Chr,Start,End,Ref,Alt,Variant_Classification,DOMAINS,...,MetaLR_score,MetaLR_pred,integrated_fitCons_score,integrated_confidence_value,GERP++_RS,phyloP7way_vertebrate,phyloP20way_mammalian,phastCons7way_vertebrate,phastCons20way_mammalian,SiPhy_29way_logOdds
0,USP24,ENSG00000162402,TCGA-D8-A1XY-01A-11D-A14K-09,chr1,55159655,55159655,T,C,Missense_Mutation,NaN,...,0.004,T,0.707,0,5.83,0.991,0.964,0.975,0.994,16.214
1,ERICH3,ENSG00000178965,TCGA-D8-A1XY-01A-11D-A14K-09,chr1,74571494,74571494,C,T,Missense_Mutation,PROSITE_profiles:PS50313,...,0.027,T,0.487,0,4.07,-0.024,0.768,0.002,0.008,11.513
2,KIF26B,ENSG00000162849,TCGA-D8-A1XY-01A-11D-A14K-09,chr1,245419680,245419680,G,T,Silent,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USP34,ENSG00000115464,TCGA-D8-A1XY-01A-11D-A14K-09,chr2,61189055,61189055,G,C,Silent,Low_complexity_(Seg):Seg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ANTXR1,ENSG00000169604,TCGA-D8-A1XY-01A-11D-A14K-09,chr2,69245305,69245305,G,A,Silent,Low_complexity_(Seg):Seg;Prints_domain:PR01217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
